# Постановка задачи

## Task #1

**Цель**
Обучаем сеть корректной последовательности действий на бирже: ожидание покупки -> покупка -> ожидание продажи -> продажа.
 
**Observation**
 - Состояние сделки

**Действия**
 1. Ожидание момента для покупки
 2. Покупка (открытие позиции)
 3. Ожидание момента для продажи
 4. Продажа (закрытие позици)

**Награда/штраф**
Действия 1 и 2 возможны при отсутствии открытой позиции. Действия 3 и 4 допустимы только при открытой позиции. При нарушении этого требования - сеть штрафуется.
 
Сеть получает награду при закрытии позиции.

# Импорты

In [1]:
# Системные импорты и настройки
import os
import sys
import yaml
import random
import warnings
import ipynbname
import logging.config

warnings.filterwarnings('ignore')

# for local development
RT_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/rt_libs/src"
BA_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/basic_application/src"
sys.path.append(RT_LIBS_PATH)
sys.path.append(BA_LIBS_PATH)

# read config
with open('config.yaml', "r") as stream:
    config = yaml.safe_load(stream)
    
# set logging config
log_config = config.get("log", None)
logging.config.dictConfig(log_config)

# set notebook alias
ALIAS = ipynbname.name()
print(ALIAS)

gen12.1-rnn-Abstract-01-SequenceTraining


In [2]:
# DS frameworks
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

%matplotlib notebook

In [3]:
# NN Frameworks
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D, AveragePooling1D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.models import load_model, clone_model

devices = tf.config.list_physical_devices()
print(devices)

2023-06-20 10:54:56.625983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [4]:
# RT packages
from rl import DQNAgent
from env import TradeEnv
from core import ConstructorGen2

from data_point import DataPointFactory

from train_tools import Player, plot_and_go

from train_tools import Player, plot_and_go
from train_tools.train_plot import TrainPlot4
from train_tools.train_manager import TrainManager

In [5]:
seed_value= 0
#os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
#np.random.seed(seed_value)
#tf.random.set_seed(seed_value)

# Конфиг

In [6]:
observation_len = 1

# Параметры точки наблюдения
observation_config = {
    "observation_len": observation_len,             # Количество точек наблюдения в сэмпле
    "offset": observation_len,                      # Количество точек наблюдения в сэмпле
    "future_points": 0,                             # Количество будущех точек для предсказания тренда (временное решение)
    "step_size": 1,                                 # Шаг по датасету
 }

# Датасет

In [7]:
n_steps = 100
data_train = pd.DataFrame(np.ones(n_steps), columns=["feature"], dtype=np.int8)
display(data_train.shape)
display(data_train.head(3))

(100, 1)

,feature
0,1
1,1
2,1


# Инициализация компонентов

## Datapoint factory

In [8]:
dpf_train = DataPointFactory(dataset=data_train, **observation_config)
dpf_test = DataPointFactory(dataset=data_train, **observation_config)

## Env

In [9]:
core_config = {
    "context":{"class": "Context", "params":{}},
    "action_controller":{
        "class": "AbstractSequencePrediction",
        "params":{ 
            "penalty": -2, 
            "reward": 0,
        },
    },
    "observation_builder":{
        "class": "ObservationBuilderInput",
        "inputs": [
            {"class": "Input1D", "features": [{"class": "TradeStateFeature", "params": {}}]
            }
        ]
    }
}
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
core_constructor = ConstructorGen2()
env_core = core_constructor.get_core(core_config)

# train environment
env = TradeEnv(env_core, dpf_train, alias=ALIAS, log=False, log_obs=False)

# Нейронная сеть

In [11]:
ACTIVATION = 'tanh'
def create_q_model(env):
    num_actions = env.action_space
    #----------------------------------------------
    
    inp_static = Input(shape=(1,env.observation_space[0]))
    classif = LSTM(8, activation=ACTIVATION, return_sequences=False)(inp_static)

    output = Dense(num_actions, activation='softmax')(classif)

    model = Model(inputs=inp_static, outputs=output)
    return model

model = create_q_model(env)
model_target = create_q_model(env)

print(model.summary())

2023-06-20 10:55:06.062399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 1)]            0         
                                                                 
 lstm (LSTM)                 (None, 8)                 320       
                                                                 
 dense (Dense)               (None, 4)                 36        
                                                                 
Total params: 356
Trainable params: 356
Non-trainable params: 0
_________________________________________________________________
None


# Обучение

In [16]:
random.seed(seed_value)

core_train = core_constructor.get_core(core_config)
core_test = core_constructor.get_core(core_config)
env = TradeEnv(core_train, dpf_train, alias=ALIAS, log=False, log_obs=False)

model = create_q_model(env)
model_target = create_q_model(env)

agent = DQNAgent(env, model, model_target)

agent.epsilon_greedy_frames = 2000
agent.epsilon_random_frames = int(0.05 * agent.epsilon_greedy_frames)
agent.max_memory_length = int(1.0 * agent.epsilon_greedy_frames)

agent.max_steps_per_episode = 50000

agent.gamma = 0.95
agent.epsilon_min = 0.01
agent.batch_size = 32
agent.update_after_actions = 4
agent.update_target_network = 250
agent.loss_function = tf.keras.losses.Huber() #tf.keras.losses.MeanSquaredError()

learning_rate = 0.0005
agent.optimizer = Adam(learning_rate=learning_rate, clipnorm=0.001)    #Adam(learning_rate=learning_rate) RMSprop(learning_rate=learning_rate) SGD(learning_rate=learning_rate)


tp = TrainPlot4()
tm = TrainManager(agent, core_test, dpf_test, tp, alias=ALIAS)

In [17]:
tp.init_plot(width=1000, height=800)
tp.update_plot(tm.history)

FigureWidget({
    'data': [{'legendgroup': '1',
              'line': {'color': '#109618', 'width': 1},
              'mode': 'lines',
              'name': 'Train',
              'type': 'scatter',
              'uid': '5fa0a86a-9766-4d0e-a883-fe2b8fd50572',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'legendgroup': '1',
              'line': {'color': '#FF9900', 'width': 1},
              'mode': 'lines',
              'name': 'Test',
              'type': 'scatter',
              'uid': 'd150ae01-912b-47f7-b25f-89ced10d2b47',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'legendgroup': '2',
              'line': {'color': '#D62728', 'width': 1},
              'mode': 'lines',
              'name': 'Train',
              'type': 'scatter',
              'uid': '54e2e8e6-99e9-446e-acd0-9c8ad4f70738',
              'xaxis': 'x2',
              'yaxis': 'y3'},
             {'legendgroup': '2',
              'line': {'color': '#FF9900'

In [18]:
tm.go(max_frames=5000, test_every=100, snapshot_every=500000, update_plot_every=100, save_since=0.06)

11:01:34 Running reward: -92.50   at episode 3    | frame 250    | eps: 0.88 | Running loss: 0.84209
11:01:37 Running reward: -82.80   at episode 6    | frame 500    | eps: 0.75 | Running loss: 0.79794
11:01:40 Running reward: -85.29   at episode 8    | frame 750    | eps: 0.63 | Running loss: 0.79133
11:01:43 Running reward: -94.00   at episode 11   | frame 1000   | eps: 0.50 | Running loss: 0.80614
11:01:46 Running reward: -98.17   at episode 13   | frame 1250   | eps: 0.38 | Running loss: 0.81735
11:01:49 Running reward: -83.80   at episode 16   | frame 1500   | eps: 0.26 | Running loss: 0.81541
11:01:53 Running reward: -75.41   at episode 18   | frame 1750   | eps: 0.13 | Running loss: 0.80152
11:01:56 Running reward: -60.90   at episode 21   | frame 2000   | eps: 0.01 | Running loss: 0.77411
11:02:00 Running reward: -51.09   at episode 23   | frame 2250   | eps: 0.01 | Running loss: 0.75495
11:02:03 Running reward: -39.12   at episode 26   | frame 2500   | eps: 0.01 | Running loss

# Итоги

Наиболее простая задача из всех.

Сеть обучилась эффективному алгоримту - открывает и сразу закрывает сделку, без ожидания. Чем больше количество закрытых сделок, тем выше совокупная награда.

Протестировал разные learning rate. При маленьких зщначениях (0.00025) обучается дольше, по мере роста  (0.001 и выше) - быстрее вызходит на оптимальный результат. При высоком значении (0.5) алгоритм начало колбасить. 

Увеличение глубины сети компенсирует длительность обучения при маленькой learning rate - с добавлением следующего слоя скорость обучения возрастает. Но процес не линейный - с определенного момента длительность обучения начинает увеличиваться (затухание градиента?)

Увеличение сложности сети (кол-ва параметров, не слоев) дает такой же эффект - обучается быстрее и стабильнее. И здесь все линейно - даже при увеличении количества нейронов на 2 порядка обучается стабильно. Снижение скорости обучения будет заметно на большой сети/наборе данных.



----


При одном слое из 4-х нейронов к концу обучения не выходила на оптимальнsй результат. При увеличении кол-ва нейронов или глубины алгоритм стал сходиться.


При lr=0.00025 на оптимум сеть вышла после 5-7 тыс фреймов. Если поставить lr выше (например 0.005) то выход на оптимум происходим быстрее - 

- 4A
    - 0.00025
        - 1200 и 3200
        - 7100
        - 4100, штрафы ушли с 1000
        - 3500, штрафы ушли с 1400
        - 2500, штрафы ушли с 2600
        - 2400, штрафы ушли с 2500
    - 0.0005
        - 400, штрафы ушли с 400
        - 900, штрафы ушли с 900
        - 2500, штрафы ушли с 900
        - 900, штрафы ушли с 200
        - 1300, штрафы ушли с 1300
        
    - 0.001
        - 1800, штрафы ушли с 500
        - 300, штрафы ушли с 300
        - 1600, штрафы ушли с 700
        - 800, штрафы ушли с 800
        - 1500, штрафы ушли с 800
        
    - 0.002
        - 1200, штрафы ушли с 300
        - 200, штрафы ушли с 100
        - 300, штрафы ушли с 300
        - 300, штрафы ушли с 300
        - 400, штрафы ушли с 400
        
    - 0.005
        - 300, штрафы ушли с 100
        - 100, штрафы ушли с 100
        - 300 и 1000, штрафы ушли с 300
        - 200, штрафы ушли с 200
        - 1000, штрафы ушли с 400
    - 0.5 - нестабильно
    